In [10]:
# %pip install sqlalchemy En caso de ser necesario, ejecutar
# %pip install --user psycopg2 En caso de ser necesario, ejecutar
# %pip install numpy En caso de ser necesario, ejecutar
# %pip install pandas En caso de ser necesario, ejecutar

In [11]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [12]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlFuente = f"{configFuente['driver']}://{configFuente['user']}:{configFuente['password']}@{configFuente['host']}:{configFuente['port']}/{configFuente['db']}"
urlBodega = f"{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}"

src = create_engine(urlFuente)
etl = create_engine(urlBodega)

In [13]:
mensajero1 = pd.read_sql_table('clientes_mensajeroaquitoy', src)
mensajero2 = pd.read_sql_table('auth_user', src)
mensajero1.drop(columns=['id', 'fecha_entrada', 'fecha_salida', 'salario', 'ciudad_operacion_id', 'token_Firebase', 'url_foto', 'activo'], inplace=True)
mensajero2.drop(columns=['password', 'last_login', 'is_superuser', 'is_staff', 'is_active', 'date_joined'], inplace=True)
mensajero = mensajero1.merge(mensajero2, left_on='user_id', right_on='id', how='left')
mensajero.drop(columns=['user_id'], inplace=True)
mensajero.rename(columns={
  'id': 'key_dim_mensajero',
  'first_name': 'nombre',
  'last_name' : 'apellido'}, inplace=True)

# Mover la columna 'telefono' al final
cols = [col for col in mensajero.columns if col != 'telefono'] + ['telefono']
mensajero = mensajero[cols]


mensajero

,key_dim_mensajero,username,nombre,apellido,email,telefono
0,2,mensajero1,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
1,410,JPEDROZA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
2,447,JULIANVILLANUEVA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
3,400,LUISCARDONA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
4,331,GEOVANNY Hidalgo,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
5,346,JHONMUÑOZ,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
6,364,JONATANMANZANO,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
7,368,LUISGIL,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
8,327,Luis Castro,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000
9,338,Vladimir Putin,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,310-300000


In [14]:
mensajero.to_sql('dim_mensajero', etl, index=False,  if_exists='replace')

50